In [1]:
# dependencies
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, join, outerjoin, MetaData, Table


In [2]:
# create engine to hawaii.sqlite
connect_string = "sqlite:///static/data/climateDB.db"

# reflect the tables
engine = create_engine(connect_string) 

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect =True)


In [3]:
Base.classes.keys()

['CO2_emission', 'country_demo', 'temp_change']

In [4]:
# Save references to each table
Emission = Base.classes.CO2_emission
Temp_change = Base.classes.temp_change

In [5]:
# Create a session (link) from Python to the sqlite DB
session = Session(bind = engine)

#Filter the data for the year >= 1961
results_emission = session.query(Emission).filter(Emission.Year >= 1961)
#print(results) 

In [6]:
emission_df = pd.read_sql(results_emission.statement, session.connection())

print(emission_df.head())
results_temp = session.query(Temp_change)

#print(results) 

temp_df = pd.read_sql(results_temp.statement, session.connection())

selection = ['DecJanFeb', 'MarAprMay', 'JunJulAug', 'SepOctNov']

                       Entity  Code  Year  AnnualCO2emissions
0                      Africa  None  1961           161437399
1                        Asia  None  1961          1392745886
2  Asia (excl. China & India)  None  1961           721204193
3                       EU-27  None  1961          2157634122
4                       EU-28  None  1961          2746248923


In [7]:




season_df = temp_df.loc[temp_df["Months"].isin(selection)]

#filter by months
month_df = temp_df.loc[(~temp_df["Months"].isin(selection)) & (temp_df["Months"] != 'Meteorological year') ]

# filter Meteorological year
meteor_df = temp_df.loc[temp_df["Months"] == 'Meteorological year' ]
# Calculate avg temp per Meteorological year
meteor_df['avg_temp']= meteor_df.mean(axis =1)
meteor_id_df = meteor_df.set_index('Area')

session.close() 

C:\Users\Courtney\anaconda3\envs\PythonData2\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [8]:
month_df["Area"].tolist()



['Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Afghanistan',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Albania',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Algeria',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andorra',
 'Andorra',
 'Angola',
 'Angola',
 'Angola',
 'Angola',
 'Angola',
 'Angola',
 'Angola',
 'Angola',
 'Angola',
 'Angola',
 'Angola',
 'Angola',
 'Anguilla',
 'Anguilla',
 'Anguilla',
 'Anguilla',
 'Anguilla',
 'Anguilla',
 'Anguilla',
 'Anguilla',
 'Anguilla',
 'Anguilla',
 'Anguilla',
 'Anguilla',
 'Antarctica',
 'Antarctica',
 'Antarctica',
 'Antarctica',
 'Antarctica',
 'Antarctica'

In [9]:
country='United States of America'

#Get avg_temp change by season for the selected country
months_country_mean = month_df.loc[month_df['Area'] == country] 
months_country_mean.set_index("Months" , inplace=True)
months_country_mean

,field1,Area,Element,Unit,1961,1962,1963,1964,1965,1966,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Months,,,,,,,,,,,,,,,,,,,,,
January,7888,United States of America,Temperature change,°C,0.943,-0.771,-1.170,0.901,0.747,-1.654,...,0.512,0.616,1.548,2.060,1.913,2.033,2.675,2.128,1.600,1.866
February,7890,United States of America,Temperature change,°C,1.215,1.425,0.577,-1.032,-1.287,-0.969,...,-0.215,-0.048,2.438,0.662,-0.347,0.389,3.788,3.528,1.765,0.375
March,7892,United States of America,Temperature change,°C,0.656,-1.082,1.400,-1.379,-1.034,0.214,...,1.211,1.251,3.688,0.162,0.220,2.532,3.778,1.844,1.632,1.404
April,7894,United States of America,Temperature change,°C,-1.210,0.409,0.399,-0.197,0.612,-0.595,...,1.414,0.546,2.043,-1.015,0.760,1.535,1.998,1.978,-0.414,1.411
May,7896,United States of America,Temperature change,°C,-0.348,0.671,0.503,0.023,-0.127,-0.389,...,0.323,-0.124,1.414,0.141,0.919,1.044,0.787,0.521,2.538,0.307
June,7898,United States of America,Temperature change,°C,0.339,-0.306,-0.165,-0.158,-0.980,-0.122,...,1.001,0.709,1.057,1.369,0.444,1.632,1.773,1.100,1.480,0.677
July,7900,United States of America,Temperature change,°C,-0.313,-0.515,-0.089,0.363,-0.622,0.562,...,0.501,1.089,1.335,0.536,-0.118,0.371,1.078,1.259,1.159,1.045
August,7902,United States of America,Temperature change,°C,0.190,0.203,-0.072,-0.645,-0.576,-0.618,...,1.127,1.292,0.870,0.631,0.370,0.445,1.093,0.100,0.738,1.018
September,7904,United States of America,Temperature change,°C,-0.467,-0.511,0.814,-0.411,-1.037,-0.005,...,1.055,0.830,0.695,0.882,0.827,1.561,1.295,0.918,1.638,2.021


In [10]:
#Drop unwanted fields
months_country_mean = months_country_mean.drop(['field1','Area', 'Element', 'Unit'], 1)
#Get years data
year = months_country_mean.columns
months_country_mean

,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Months,,,,,,,,,,,,,,,,,,,,,
January,0.943,-0.771,-1.170,0.901,0.747,-1.654,1.635,-0.081,-0.487,-1.646,...,0.512,0.616,1.548,2.060,1.913,2.033,2.675,2.128,1.600,1.866
February,1.215,1.425,0.577,-1.032,-1.287,-0.969,-0.367,-0.567,-0.438,1.474,...,-0.215,-0.048,2.438,0.662,-0.347,0.389,3.788,3.528,1.765,0.375
March,0.656,-1.082,1.400,-1.379,-1.034,0.214,1.739,1.691,-1.689,-0.264,...,1.211,1.251,3.688,0.162,0.220,2.532,3.778,1.844,1.632,1.404
April,-1.210,0.409,0.399,-0.197,0.612,-0.595,0.400,-0.217,1.149,-0.656,...,1.414,0.546,2.043,-1.015,0.760,1.535,1.998,1.978,-0.414,1.411
May,-0.348,0.671,0.503,0.023,-0.127,-0.389,-1.053,-0.817,0.764,0.461,...,0.323,-0.124,1.414,0.141,0.919,1.044,0.787,0.521,2.538,0.307
June,0.339,-0.306,-0.165,-0.158,-0.980,-0.122,-0.200,-0.018,-0.440,0.037,...,1.001,0.709,1.057,1.369,0.444,1.632,1.773,1.100,1.480,0.677
July,-0.313,-0.515,-0.089,0.363,-0.622,0.562,-0.510,-0.062,0.080,0.020,...,0.501,1.089,1.335,0.536,-0.118,0.371,1.078,1.259,1.159,1.045
August,0.190,0.203,-0.072,-0.645,-0.576,-0.618,-0.246,-0.374,0.094,0.454,...,1.127,1.292,0.870,0.631,0.370,0.445,1.093,0.100,0.738,1.018
September,-0.467,-0.511,0.814,-0.411,-1.037,-0.005,-0.287,-0.538,0.479,-0.430,...,1.055,0.830,0.695,0.882,0.827,1.561,1.295,0.918,1.638,2.021


In [11]:
months_list = []
# Get months for each country
month_index = months_country_mean.index
month_index

#Create an object to hold keys[]
months_obj ={
        'Country': country,       
            }

if len(months_country_mean) > 4:
    months_obj['Year'] =list(np.ravel(year))
    for item in month_index:
        months_obj[item] = list(np.ravel(months_country_mean.loc[item].values)),
    #print(country_df.loc[item].values)
    months_obj["Data Found"] = 'Yes'    
    
else:
    months_obj["Data Found"] = 'No' 
    #print(f'...............\n Country: {country}\n')
    #print(country_df)
    #print(months_obj)

months_list.append(months_obj)
        
        
       
        
months_list


[{'Country': 'United States of America',
  'Year': ['1961',
   '1962',
   '1963',
   '1964',
   '1965',
   '1966',
   '1967',
   '1968',
   '1969',
   '1970',
   '1971',
   '1972',
   '1973',
   '1974',
   '1975',
   '1976',
   '1977',
   '1978',
   '1979',
   '1980',
   '1981',
   '1982',
   '1983',
   '1984',
   '1985',
   '1986',
   '1987',
   '1988',
   '1989',
   '1990',
   '1991',
   '1992',
   '1993',
   '1994',
   '1995',
   '1996',
   '1997',
   '1998',
   '1999',
   '2000',
   '2001',
   '2002',
   '2003',
   '2004',
   '2005',
   '2006',
   '2007',
   '2008',
   '2009',
   '2010',
   '2011',
   '2012',
   '2013',
   '2014',
   '2015',
   '2016',
   '2017',
   '2018',
   '2019'],
  'January': ([0.943,
    -0.771,
    -1.17,
    0.901,
    0.747,
    -1.654,
    1.635,
    -0.081,
    -0.487,
    -1.646,
    -1.209,
    -0.216,
    -0.504,
    0.538,
    0.798,
    0.146,
    -1.311,
    -0.435,
    -2.283,
    0.542,
    2.946,
    -1.677,
    1.42,
    -0.066,
    0.274,
   

In [14]:
group_temp = temp_df.groupby('Area')
group_temp.mean()

,field1,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Area,,,,,,,,,,,,,,,,,,,,,
Afghanistan,16.0,0.027941,-0.197471,0.888706,-0.905647,-0.051824,0.222118,-0.362176,-0.349294,-0.377882,...,1.499235,1.246118,0.179765,1.251706,0.487000,1.098294,1.671882,1.306765,1.574706,0.899588
Africa,8448.0,-0.089000,-0.006353,0.077471,-0.189471,-0.195294,0.146706,-0.222588,-0.181647,0.350294,...,1.518059,0.892941,0.814941,1.002353,1.049176,1.207941,1.483941,1.188647,1.236059,1.428000
Albania,50.0,0.473235,0.238941,0.254647,-0.197118,-0.361588,0.460294,-0.078353,0.060765,-0.037765,...,1.208706,1.137765,1.513235,1.541706,1.466529,1.689471,1.616000,1.416529,2.206412,2.029000
Algeria,84.0,0.302059,0.029706,0.176824,0.056529,-0.062353,0.320412,-0.072647,0.038000,0.183000,...,2.334000,1.354294,1.258824,1.224412,1.772529,1.285235,1.910765,1.518588,1.404353,1.289529
Americas,8652.0,-0.005941,-0.017941,0.212412,-0.309176,-0.207765,-0.128235,-0.071000,-0.062059,0.228765,...,1.356235,0.904000,1.319059,0.787765,0.843647,1.397941,1.664706,1.381294,0.967529,1.247471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Western Europe,9162.0,0.905824,-0.680941,-0.984882,0.006294,-0.585000,0.392588,0.525824,0.005765,-0.164765,...,0.154412,1.697235,1.203235,0.733647,2.220765,1.846471,1.657059,1.707118,2.292941,2.160176
Western Sahara,8244.0,0.694941,0.410706,0.167941,0.702882,-0.325118,0.266471,-0.031941,-0.188176,0.284353,...,1.931706,1.452000,1.341235,1.420059,1.278353,1.607471,1.619588,2.094588,0.922059,1.597294
World,8414.0,0.155941,0.053235,0.139000,-0.277353,-0.170941,0.171412,-0.113706,-0.112000,-0.057941,...,1.229824,0.948471,1.036118,1.065647,1.068706,1.460647,1.626059,1.462353,1.278294,1.510882
